In [103]:
import io
import xml.etree.ElementTree as ET
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import numpy as np
import os
from gensim import corpora, models, similarities 
from gensim.corpora.mmcorpus import MmCorpus
import helper
import corpora
import model
import pyLDAvis.gensim
import operator 
import model_exporter
import logging


time: 9.49 ms


In [104]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 296 ms


In [105]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 952 ms


In [106]:
#%unload_ext autotime

time: 1.1 ms


[autoreload of autotime failed: Traceback (most recent call last):
  File "/home/npatta01/.virtualenvs/pyspark/local/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named autotime
]


In [107]:
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

fileHandler=logging.FileHandler("log.log")
logging.root.addHandler(fileHandler) 




time: 182 ms


# Working Directory

In [108]:
pwd

u'/home/npatta01/projects/metis_fletcher/analysis'

time: 59 ms


Super User Forum data is in data folder

In [109]:
ls ../data

data.sqlite  makedb.py  Posts.xml*  stopwords.txt
time: 207 ms


In [110]:
data_dir='data'

time: 1.57 ms


## Normalizer

A wrapper class that allows me to easily change/apply steeming options

In [111]:
options = {
    "stem":False
    ,"stop_words":"../data/stopwords.txt"
    ,"strip_html_tags":True
    ,"filter_parts_of_speech":True
    ,"min_word_length":2
    ,"lower_case":True
    
}

time: 130 ms


In [112]:
ag_preprocesser = helper.Normalizer(**options)

time: 78.6 ms


Example of the text processor being applied

In [113]:
sample_msg='<p>A Vista virtual machine I use only has a 10 GB virtual hard disk -- and I\'m worried about it running out of space over time.</p>\n\n<p>I used <a href="http://www.jam-software.com/treesize_free/">TreeSize</a> to check for the culprits..</p>\n\n<p><img src="http://i.stack.imgur.com/uGfhQ.png" alt="enter image description here"></p>\n\n<p>And the primary culprit is that <strong><code>/winsxs</code> or "Windows Side-By-Side" folder</strong>.</p>\n\n<p><img src="http://i.stack.imgur.com/DWYOo.png" alt="enter image description here"></p>\n\n<p>What is the Windows Side By Side folder? Well, that\'s <a href="http://blogs.technet.com/askcore/archive/2008/09/17/what-is-the-winsxs-directory-in-windows-2008-and-windows-vista-and-why-is-it-so-large.aspx">explained pretty well here</a>.</p>\n\n<blockquote>\n  <p>All of the components in the operating system are found in the WinSxS folder \u2013 in fact we call this location the component store.  Each component has a unique name that includes the version, language, and processor architecture that it was built for.  The WinSxS folder is the only location that the component is found on the system, all other instances of the files that you see on the system are \u201cprojected\u201d by hard linking from the component store.  Let me repeat that last point \u2013 there is only one instance (or full data copy) of each version of each file in the OS, and that instance is located in the WinSxS folder.   So looked at from that perspective, the WinSxS folder is really the entirety of the whole OS, referred to as a "flat" in down-level operating systems.  This also accounts for why you will no longer be prompted for media when running operations such as System File Checker (SFC), or when installing additional features and roles.</p>\n  \n  <p>That explains why the folder starts off big, but not why it gets larger over time \u2013 the answer to that question is servicing.   In previous versions of Windows the atomic unit of servicing was the file, in Windows Vista it\u2019s the component.  When we update a particular binary we release a new version of the whole component, and that new version is stored alongside the original one in the component store.  The higher version of the component is projected onto the system, but the older version in the store isn\u2019t touched.  The reason for that is the third part of why the component store gets so large.</p>\n</blockquote>\n\n<p>Lots of other people <a href="http://blog.tiensivu.com/aaron/archives/1306-Demystifying-the-WinSxS-directory-in-Windows-XP,-Vista-and-Server-20032008.html">complaining about the size of the /winsxs folder, too</a>:</p>\n\n<blockquote>\n  <p>A coworker recently received his every two year work laptop refresh and was in the middle of loading his new laptop with Vista. For some reason, his hard drive C: was partitioned to only 25GB and the rest of the space reserved for a much larger drive D:.</p>\n  \n  <p>Anyway, after loading many Windows Updates, he was left with hardly any drive space left on C:, and for some reason, the WinSxS directory had ballooned to over 8GB.</p>\n  \n  <p>While I knew what the WinSxS (Windows Side-by-Side) directory was for, I wasn\'t quite sure why that directory would explode in size so fast with normal operating system updates.</p>\n</blockquote>\n\n<p>I don\'t really care about having all these duplicate files for safety, but I <em>do</em> care about the VM exploding due to an out of disk space error. So.. <strong>can /winsxs be safely pared down in any way?</strong> <a href="http://blogs.msdn.com/jonwis/archive/2007/01/02/deleting-from-the-winsxs-directory.aspx">It doesn\'t seem that it can be</a>.. </p>\n\n<blockquote>\n  <p><strong>Administrators should not, for any reason, take it upon themselves to clean out the /winsxs directory</strong> - doing so may prevent Windows Update and MSI from functioning properly afterwards.  Preventing accidental deletion from the directory is accomplished by putting a strong security descriptor on the directory that inherits to its children.</p>\n</blockquote>\n\n<p>If this /winsxs folder gets any larger, I\'m going to have to re-image this virtual machine.. which I <em>really</em> don\'t want to do!</p>\n'

ag_preprocesser.process(sample_msg)

[u'vista',
 u'virtual',
 u'machine',
 u'use',
 u'virtual',
 u'hard',
 u'disk',
 u'worried',
 u'running',
 u'space',
 u'time',
 u'used',
 u'treesize',
 u'check',
 u'culprits..',
 u'primary',
 u'culprit',
 u'/winsxs',
 u'windows',
 u'side-by-side',
 u'folder',
 u'windows',
 u'side',
 u'side',
 u'folder',
 u'well',
 u'explained',
 u'pretty',
 u'well',
 u'components',
 u'operating',
 u'system',
 u'found',
 u'winsxs',
 u'folder',
 u'\\u2013',
 u'fact',
 u'call',
 u'location',
 u'component',
 u'store',
 u'component',
 u'unique',
 u'name',
 u'includes',
 u'version',
 u'language',
 u'processor',
 u'architecture',
 u'built',
 u'winsxs',
 u'folder',
 u'location',
 u'component',
 u'found',
 u'system',
 u'instances',
 u'files',
 u'see',
 u'system',
 u'\\u201cprojected\\u201d',
 u'hard',
 u'linking',
 u'component',
 u'store',
 u'let',
 u'repeat',
 u'last',
 u'point',
 u'\\u2013',
 u'one',
 u'instance',
 u'full',
 u'data',
 u'copy',
 u'version',
 u'file',
 u'instance',
 u'located',
 u'winsxs',
 u'fo

time: 105 ms


## Load Streaming corpora

Raw xml corpora was converted to a sqlite database by running the script makedb.py

In [114]:
db_path ='../data/data.sqlite'

time: 65.8 ms


In [115]:
posts_corpus= corpora.SuperUserSqliteCorpus(
    db_path=db_path
    ,normalizer=ag_preprocesser)
posts_corpus.finalize_dictionary()


--- Completed 257,000 docs 

INFO:gensim.corpora.dictionary:discarding 528314 tokens: [(u'culprits..', 1), (u'\u201cprojected\u201d', 1), (u'ballooned', 2), (u'down-level', 2), (u'win7rc', 2), (u'fileboxextender', 1), (u'aaa-webs', 1), (u'bbb-audio', 1), (u'sytax', 3), (u'vam', 1)]...
INFO:gensim.corpora.dictionary:keeping 50769 tokens which were in no less than 5 and no more than 128559 (=50.0%) documents
INFO:gensim.corpora.dictionary:resulting dictionary: Dictionary(50769 unique tokens: [u'/usr/share/fonts/x11/type1', u'sub2.example.com', u'mdraid', u'userscripts', u'winclone']...)



time: 17min 29s


Example of streaming over xml corpora

In [116]:
"""
post_file=os.path.join(data_dir,'Posts.xml')
posts_corpus = helper.XmlCorpus(post_file
                                ,clip_docs=100000
                                #,clip_docs=1000
                                ,body_field='Body'
                                ,normalizer=ag_preprocesser)
posts_corpus.finalize_dictionary()
"""

"\npost_file=os.path.join(data_dir,'Posts.xml')\nposts_corpus = helper.XmlCorpus(post_file\n                                ,clip_docs=100000\n                                #,clip_docs=1000\n                                ,body_field='Body'\n                                ,normalizer=ag_preprocesser)\nposts_corpus.finalize_dictionary()\n"

time: 3.06 ms


In [117]:
print ("Number of docs in collection: %s"%(len(posts_corpus)))

Number of docs in collection: 257118
time: 220 ms


# Building Gensim Model

In [118]:
num_topics = 8
passes=10
eval_every=0

time: 76.6 ms


In [119]:
lda = gensim.models.LdaMulticore(posts_corpus, num_topics=num_topics, 
                            id2word=posts_corpus.dictionary
                                 ,passes=passes, eval_every=eval_every
                                )

INFO:gensim.models.ldamodel:using symmetric alpha at 0.125
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamulticore:running online LDA training, 8 topics, 10 passes over the supplied corpus of 257118 documents, updating every 10000 documents, evaluating every ~0 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamulticore:training LDA model using 5 processes


--- Completed 2,000 docs 

INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/257118, outstanding queue size 1


--- Completed 4,000 docs 

INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/257118, outstanding queue size 2


...

--- Completed 256,000 docs 

INFO:gensim.models.ldamulticore:PROGRESS: pass 9, dispatched chunk #127 = documents up to #256000/257118, outstanding queue size 2


--- Completed 257,000 docs 

INFO:gensim.models.ldamulticore:PROGRESS: pass 9, dispatched chunk #128 = documents up to #257118/257118, outstanding queue size 2
INFO:gensim.models.ldamodel:merging changes from 7118 documents into a model of 257118 documents
INFO:gensim.models.ldamodel:topic #6 (0.125): 0.016*would + 0.014*like + 0.013*data + 0.012*excel + 0.010*one + 0.010*want + 0.009*need + 0.009*use + 0.007*column + 0.007*way
INFO:gensim.models.ldamodel:topic #7 (0.125): 0.021*error + 0.018*command + 0.015*file + 0.012*root + 0.010*script + 0.009*run + 0.007*directory + 0.007*version + 0.006*user + 0.006*install
INFO:gensim.models.ldamodel:topic #4 (0.125): 0.035*windows + 0.013*user + 0.010*folder + 0.008*account + 0.007*password + 0.007*use + 0.007*server + 0.007*using + 0.007*access + 0.007*files
INFO:gensim.models.ldamodel:topic #0 (0.125): 0.039*file + 0.024*files + 0.013*text + 0.011*like + 0.010*want + 0.009*using + 0.008*way + 0.008*word + 0.007*folder + 0.007*line
INFO:gensim.models.ldamodel:topic #3 (0


time: 3h 2min 35s


Topics Learned

In [120]:
lda.show_topics(num_topics)

[u'0.039*file + 0.024*files + 0.013*text + 0.011*like + 0.010*want + 0.009*using + 0.008*way + 0.008*word + 0.007*folder + 0.007*line',
 u'0.023*network + 0.022*server + 0.015*router + 0.014*connection + 0.013*connect + 0.011*address + 0.011*internet + 0.010*port + 0.010*access + 0.009*using',
 u'0.020*windows + 0.012*screen + 0.010*problem + 0.008*time + 0.008*chrome + 0.007*tried + 0.006*computer + 0.006*like + 0.006*window + 0.006*using',
 u'0.012*card + 0.012*device + 0.012*usb + 0.011*driver + 0.010*video + 0.010*laptop + 0.009*ram + 0.009*cpu + 0.009*memory + 0.009*power',
 u'0.035*windows + 0.013*user + 0.010*folder + 0.008*account + 0.007*password + 0.007*use + 0.007*server + 0.007*using + 0.007*access + 0.007*files',
 u'0.032*drive + 0.027*windows + 0.019*disk + 0.019*boot + 0.014*partition + 0.013*hard + 0.011*system + 0.010*usb + 0.008*install + 0.008*linux',
 u'0.016*would + 0.014*like + 0.013*data + 0.012*excel + 0.010*one + 0.010*want + 0.009*need + 0.009*use + 0.007*colu

time: 19.2 ms


Topics without words frequency

In [121]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,:,1]
for i in topic_words:
    print([str(word) for word in i])
    print()

['file', 'files', 'text', 'like', 'want', 'using', 'way', 'word', 'folder', 'line', 'use', 'would', 'one', 'open', 'find', 'command', 'copy', 'directory', 'page', 'document']
()
['network', 'server', 'router', 'connection', 'connect', 'address', 'internet', 'port', 'access', 'using', 'host', 'ssh', 'wireless', 'use', 'connected', 'remote', 'wifi', 'machine', 'local', 'dns']
()
['windows', 'screen', 'problem', 'time', 'chrome', 'tried', 'computer', 'like', 'window', 'using', 'work', 'keyboard', 'get', 'firefox', 'back', 'open', 'way', 'issue', 'one', 'running']
()
['card', 'device', 'usb', 'driver', 'video', 'laptop', 'ram', 'cpu', 'memory', 'power', 'intel', 'drivers', 'audio', 'computer', 'graphics', 'monitor', 'motherboard', 'sound', 'one', 'use']
()
['windows', 'user', 'folder', 'account', 'password', 'use', 'server', 'using', 'access', 'files', 'way', 'want', 'install', 'computer', 'email', 'outlook', 'like', 'would', 'users', 'get']
()
['drive', 'windows', 'disk', 'boot', 'partiti

# Saving 

## Save dictioonary

In [147]:
posts_corpus_dict = posts_corpus.dictionary
dictionary_path='../visualizations/artifacts/lda.dict'
posts_corpus_dict.save(dictionary_path)

INFO:gensim.utils:saving Dictionary object under ../visualizations/artifacts/lda.dict, separately None


time: 197 ms


# Save model

In [125]:
model_serialized_path='../visualizations/artifacts/lda_topic_%s.lda'%(num_topics)
lda.save(model_serialized_path) 

INFO:gensim.utils:saving LdaState object under ../visualizations/artifacts/lda_topic_8.lda.state, separately None
INFO:gensim.utils:saving LdaMulticore object under ../visualizations/artifacts/lda_topic_8.lda, separately None
INFO:gensim.utils:not storing attribute state
INFO:gensim.utils:not storing attribute dispatcher


time: 165 ms


# Loading model

In [126]:
lda = gensim.models.LdaMulticore.load(model_serialized_path)
posts_corpus_dict = gensim.corpora.dictionary.Dictionary.load(dictionary_path)

INFO:gensim.utils:loading LdaMulticore object from ../visualizations/artifacts/lda_topic_8.lda
INFO:gensim.utils:loading id2word recursively from ../visualizations/artifacts/lda_topic_8.lda.id2word.* with mmap=None
INFO:gensim.utils:setting ignored attribute state to None
INFO:gensim.utils:setting ignored attribute dispatcher to None
INFO:gensim.utils:loading LdaMulticore object from ../visualizations/artifacts/lda_topic_8.lda.state
INFO:gensim.utils:loading Dictionary object from ../visualizations/artifacts/posts_corpus.dict


time: 296 ms


In [127]:
lda.show_topics()

[u'0.039*file + 0.024*files + 0.013*text + 0.011*like + 0.010*want + 0.009*using + 0.008*way + 0.008*word + 0.007*folder + 0.007*line',
 u'0.023*network + 0.022*server + 0.015*router + 0.014*connection + 0.013*connect + 0.011*address + 0.011*internet + 0.010*port + 0.010*access + 0.009*using',
 u'0.020*windows + 0.012*screen + 0.010*problem + 0.008*time + 0.008*chrome + 0.007*tried + 0.006*computer + 0.006*like + 0.006*window + 0.006*using',
 u'0.012*card + 0.012*device + 0.012*usb + 0.011*driver + 0.010*video + 0.010*laptop + 0.009*ram + 0.009*cpu + 0.009*memory + 0.009*power',
 u'0.035*windows + 0.013*user + 0.010*folder + 0.008*account + 0.007*password + 0.007*use + 0.007*server + 0.007*using + 0.007*access + 0.007*files',
 u'0.032*drive + 0.027*windows + 0.019*disk + 0.019*boot + 0.014*partition + 0.013*hard + 0.011*system + 0.010*usb + 0.008*install + 0.008*linux',
 u'0.016*would + 0.014*like + 0.013*data + 0.012*excel + 0.010*one + 0.010*want + 0.009*need + 0.009*use + 0.007*colu

time: 18.9 ms


# Identifying topics

In [128]:
sample_post_1="""

Can't insert cells in Excel 2010 - “operation not allowed” error message

I was working on a spreadsheet in Excel 2010, and all of a sudden when I attempted to insert a new row of cells, I saw that the insert and delete options were grayed out.

I attempted to copy a different row and insert it as a new row, but I got the error message:

"This operation is not allowed. The operation is attempting to shift cells in a table on your worksheet."

I have not merged or hidden any cells/rows/columns. There are no formulas. There is no data verification. I tried closing and re-opening the spreadsheet.

Searching for answers brings up nothing useful.
As the message indicates, you're trying to insert rows into a Table in your sheet. Typically, but not necessarily, tables will have banded formatting. When you click in a cell in a table, the Table tab will appear, like so:


"""




time: 2.73 ms


In [129]:
sample_post_2="""
I start sshd on 192.168.1.8 with the following:

sudo /usr/sbin/sshd -ddd
I then try to open an SSH tunnel to 192.168.1.5:1234 and am unable to do so. Here's what sshd -ddd says:

debug1: server_input_channel_open: confirm direct-tcpip
debug3: channel 1: waiting for connection
debug1: channel 1: connected to 192.168.1.5 port 1234
debug2: channel 1: read<=0 rd 10 len 0
debug2: channel 1: read failed
debug2: channel 1: close_read
debug2: channel 1: input open -> drain
debug2: channel 1: ibuf empty
debug2: channel 1: send eof
debug2: channel 1: input drain -> closed
debug2: channel 1: rcvd close
debug2: channel 1: output open -> drain
debug3: channel 1: will not send data after close
debug2: channel 1: obuf empty
debug2: channel 1: close_write
debug2: channel 1: output drain -> closed
debug2: channel 1: send close
debug2: channel 1: is dead
debug2: channel 1: garbage collecting
debug1: channel 1: free: direct-tcpip, nchannels 2
debug3: channel 1: status: The following connections are open:
  #0 server-session (t4 r256 i0/0 o0/0 fd 11/9 cc -1)
  #1 direct-tcpip (t4 r257 i3/0 o3/0 fd 10/10 cc -1)
So it's able to connect but the read is failing. And yet I'm able to connect (with no failing reads) when I do this (on 192.168.1.8):

ssh -p 1234 username@192.168.1.5
So why is the tunnel failing on 192.168.1.8 when the direct ssh connection on 192.168.1.8 isn't?

I'm using PuTTY to create the tunnel.



what is the exact command you type when you "open an SSH tunnel"? –  Jakuje 34 mins ago
  	 	
I'm using PuTTY to dynamically create the tunnel on-demand. PuTTY creates a SOCKS5 server that I send requests to and read responses from. I then use a PHP script I wrote to connect to the SOCKS5 server, which in turn, tells PuTTY to request the tunnel. And PuTTY is creating the tunnel, as demonstrated above. I can successfully create tunnel's to google.com:80 and am able to send an HTTP request and read the response but not so for 192.168.1.5:1234. At minimum I should be getting the SSH server identifier (SSH-2.0-whatever) but I'm not. –  neubert 26 mins ago

"""




time: 6.66 ms


prediction

In [130]:
# process tokens
processed_tokens=ag_preprocesser.process(sample_post_1)
# get word to word id
doc_bow=posts_corpus_dict.doc2bow(processed_tokens)
# get topics
sorted(lda[doc_bow],key=operator.itemgetter(1),reverse=True)

[(6, 0.52013027228366104),
 (2, 0.2300835947504449),
 (5, 0.13490280675457672),
 (0, 0.067698350890112224),
 (7, 0.041897447099740699)]

time: 14.2 ms


In [131]:
lda.show_topic(3)

[(0.012292382518809078, u'card'),
 (0.012071218106091913, u'device'),
 (0.011516879884239261, u'usb'),
 (0.010504076595377618, u'driver'),
 (0.010195595711911084, u'video'),
 (0.0096458200263295265, u'laptop'),
 (0.0094692594967971857, u'ram'),
 (0.0094556829144670108, u'cpu'),
 (0.0090597028900775492, u'memory'),
 (0.0086881361734996929, u'power')]

time: 6.38 ms


# Strongest Doc for topic

Wrapper code around sqlite corpora   

Wrapper aroundgensim lda

In [132]:
post_db= helper.SuperUserDatabase(db_path)
l_model=model.Model(corpora_processor_options=options)
l_model.load(model_serialized_path,dictionary_path)

INFO:gensim.utils:loading LdaMulticore object from ../visualizations/artifacts/lda_topic_8.lda
INFO:gensim.utils:loading id2word recursively from ../visualizations/artifacts/lda_topic_8.lda.id2word.* with mmap=None
INFO:gensim.utils:setting ignored attribute state to None
INFO:gensim.utils:setting ignored attribute dispatcher to None
INFO:gensim.utils:loading LdaMulticore object from ../visualizations/artifacts/lda_topic_8.lda.state
INFO:gensim.utils:loading Dictionary object from ../visualizations/artifacts/posts_corpus.dict


time: 134 ms


In [133]:
doc_strengths=l_model.calculate_strongest_docs(post_db)

--- Completed 257,000 
time: 22min 59s


In [134]:
doc_strengths

[{'docs': [{'answers': u'',
    'id': 158626,
    'question': u'<p>I\'m using Microsoft Word 2007.  I have several large screenshots that need to be inserted into the document at various places.  When I paste the pictures, I have to manually resize them to fit within the margins of the document.  This takes up too much of my time.  I tried batch resizing the images before inserting them, so they were the right size, but it appears Word uses a higher dpi when printing, so those pictures came out blurry compared to the ones resized within Word.</p>\n\n<p>So what I am looking for is if there is an option that makes Word automatically shrink large images when I paste them so that they are the width of the page (with correct aspect ratio).  I\'m using "in line with text," though I could use "tight" if necessary.</p>\n\n<p>Again, this isn\'t a life-or-death thing.  I could always manually resize the 30+ images.  But it seems like there should be an easier way.  Has anyone come across this?</

time: 112 ms


In [135]:
p=post_db.get_post_with_id("846129")
p.__dict__

{'answers': u'',
 'creation_date': u'2014-11-28 16:02:47.917000',
 'id': 846129,
 'question': u'<p>I\'ve got quite an interesting system configuration here, and I would like to install Windows 10 preview on it. I have 3 drives, out of which one is a SSD. They all have a GPT table.</p>\n\n<ul>\n<li><strong>Drive 0</strong> (the ssd) has the following partitions on it:\n<ul>\n<li>EFI SYSTEM PARTITION (fat32)</li>\n<li>Linux \'/\' (ext4)</li>\n<li>Linux \'/home\' (ext4)</li>\n</ul></li>\n<li><strong>Drive 1</strong> is fully dedicated to data.\n<ul>\n<li>LVM2 Physical Volume</li>\n</ul></li>\n<li><strong>Drive 2</strong> is the drive I want to install windows to.\n<ul>\n<li>LVM2 Physical Volume</li>\n<li>Empty space (for windows)</li>\n</ul></li>\n</ul>\n\n<p>After I boot the installer in UEFI mode, I select the \'Drive 2 Unallocated space\' entry. It returns an error message:</p>\n\n<blockquote>\n  <p>Your EFI system partition is formatted as NTFS. It needs to be formatted as FAT32. Plea

time: 54 ms


# Pyldavis

In [136]:
?pyLDAvis.gensim.prepare

time: 97 ms


In [137]:
pyldavis_data=pyLDAvis.gensim.prepare(lda, posts_corpus, posts_corpus_dict)

--- Completed 257,000 docs 
--- Completed 257,000 docs 
--- Completed 257,000 docs 
time: 58min 22s


In [138]:
lda_viz_path='../visualizations/static/components/pyldaviz/lda_viz.html'
pyLDAvis.save_html(pyldavis_data,lda_viz_path)

time: 24.8 ms


In [139]:
pyLDAvis.display(pyldavis_data)

time: 125 ms


# Save Results for web

In [140]:
m_exporter = model_exporter.ModelExporter(model_serialized_path)

INFO:gensim.utils:loading LdaMulticore object from ../visualizations/artifacts/lda_topic_8.lda
INFO:gensim.utils:loading id2word recursively from ../visualizations/artifacts/lda_topic_8.lda.id2word.* with mmap=None
INFO:gensim.utils:setting ignored attribute state to None
INFO:gensim.utils:setting ignored attribute dispatcher to None
INFO:gensim.utils:loading LdaMulticore object from ../visualizations/artifacts/lda_topic_8.lda.state


time: 162 ms


Save json of word count

In [141]:
topics_word_count=m_exporter.topics(num_words=50)
topics_path='../visualizations/artifacts/topics_word_count.json'
helper.save_to_json(topics_word_count,topics_path)

time: 29.9 ms


Save gensim lda model

In [142]:
lda.save(model_serialized_path) 

INFO:gensim.utils:saving LdaState object under ../visualizations/artifacts/lda_topic_8.lda.state, separately None
INFO:gensim.utils:saving LdaMulticore object under ../visualizations/artifacts/lda_topic_8.lda, separately None
INFO:gensim.utils:not storing attribute state
INFO:gensim.utils:not storing attribute dispatcher


time: 184 ms


save dictionary

In [143]:
posts_corpus_dict.save(dictionary_path)

INFO:gensim.utils:saving Dictionary object under ../visualizations/artifacts/posts_corpus.dict, separately None


time: 206 ms


save pylda viz html

In [144]:
pyLDAvis.save_html(pyldavis_data,lda_viz_path)

time: 24 ms


save frequencies

In [145]:
doc_strength_path='../visualizations/artifacts/doc_strength.json'

helper.save_to_json(doc_strengths,doc_strength_path)

time: 93.2 ms


export strongest docs

In [146]:
sample_doc_strength_path='../visualizations/artifacts/sample_docs.json'
formatted_doc_strengths=helper.export_top_docs(doc_strengths)
helper.save_to_json(formatted_doc_strengths,sample_doc_strength_path)

time: 82.6 ms
